#### Cassandra Python Programming Assignment 
##### Gayathri Shanmuga Sundaram - U88629479
INSTRUCTIONS:
1. Connect to the Cassandra Database
2. Create a keyspace called m14
3. Create a table in this keyspace that will be used to store inventory information. This will include the fields, SKU (which is shown for stock keeping unit), name (short name for the product), description (longer description of the product), warehouse_num (the warehouse number where the product is stored)
4. Populate several rows of data into your inventory table
5. Create an index on warehouse_num. 
6. Search for all products located at a given location (for instance - if you created two rows with location 123, list all products at location 123)

Importing Packages

In [ ]:
import pandas as pd
from cassandra.cluster import Cluster

Loading Data from csv file

In [ ]:
df = pd.read_csv('./toystore_inventory.csv')
df

Create a session connection to Cassandra cluster

In [ ]:
clstr = Cluster()
session = clstr.connect()

Create keyspace

In [ ]:
session.execute("CREATE KEYSPACE IF NOT EXISTS inventory WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")

Veryfing keyspace creation

In [ ]:
rows = session.execute("desc keyspaces")

In [ ]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

Create table

In [ ]:
session.execute("""
CREATE TABLE IF NOT EXISTS inventory.toystore ( 
    sku TEXT, 
    name TEXT, 
    description TEXT,  
    warehouse_num INT, 
    PRIMARY KEY(sku)
);
""")

Inserting the Row values

In [ ]:
for index, row in df.iterrows():
    session.execute(f"""
        INSERT INTO inventory.toystore (sku, name, description, warehouse_num)     
        VALUES ({row[0].strip()}, {row[1].strip()}, {row[2].strip()}, {row[3]});
        """
       )

Create View

In [ ]:
session.execute ("CREATE INDEX ON inventory.toystore(warehouse_num)")

Run the Query

In [ ]:
rows = session.execute("select (sku, name, description, warehouse_num) from inventory.toystore where warehouse_num = 6677 ALLOW FILTERING")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]},")